# NBA(미국 농구 리그) 웹사이트에서 다음 경기일정 찾기

In [8]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

## 셀레니움을 사용해 주별 경기날짜, 시간, 장소, 경기 팀, 홈팀 정보 수집

In [14]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver_path = '/home/rlxdn/Downloads/chromedriver'
driver = webdriver.Chrome(driver_path)
driver.get("https://www.nba.com/schedule?cal=October&region=1")

WebDriverWait(driver, 10).until(
  EC.presence_of_element_located(
      (By.CSS_SELECTOR, ".GameScheduleWeek_gswHeader__3i3gT")
  )
)

date = '#__next > div.Layout_justNav__3CIEq > div.max-w-screen-xl.mx-auto.p-0.md\:p-7.xxl\:px-0 > section > div > div:nth-child(3) > div:nth-child(1) > div:nth-child(2) > div.flex-grow-1.sm\:flex-grow-3.lg\:flex-grow-4 > div.flex.flex-col > div:nth-child(1) > div:nth-child(1) > div.pb-5.pr-5.min-w-1\/2.xl\:min-w-4\/10 > span'
a = driver.find_element_by_css_selector(date)
a.getText()
driver.close()

AttributeError: 'WebElement' object has no attribute 'getText'